In [1]:
import pandas as pd

In [36]:
df = pd.read_csv('bybit_BTCUSDT_20240501_20240531.csv', index_col=0)

In [37]:
# 次の5分間の最高値と最安値を計算する関数
def calculate_high_low(df, window=5):
    df['future_high'] = df['High'].rolling(window=window, min_periods=1).max().shift(-window)
    df['future_low'] = df['Low'].rolling(window=window, min_periods=1).min().shift(-window)
    return df


In [38]:
df = calculate_high_low(df)

In [39]:
# 各時刻の終値と比較し、価格が離れている方を新たな列として追加する関数
def add_most_significant_future_price(df):
    df['most_significant_future_price'] = df.apply(
        lambda row: row['future_high'] if abs(row['future_high'] - row['Close']) > abs(row['future_low'] - row['Close']) else row['future_low'],
        axis=1
    )
    return df

In [40]:
df = add_most_significant_future_price(df)

In [41]:
def add_intervening_extreme_prices(df,window=5):
    df['intervening_extreme_price'] = None
    for i in range(len(df)-window):
        row = df.iloc[i]
        minidata = df[i:i+window+1]
        minidata.reset_index(inplace=True, drop=True)
        if row['most_significant_future_price'] == row['future_high']:
            target = minidata.index[minidata['High'] == row['future_high']].tolist()[0]
            target_df = minidata[1:target]
            intervening_low = target_df['Low'].min()
            df.at[i, 'intervening_extreme_price'] = intervening_low
        else:
            target = minidata.index[minidata['Low'] == row['future_low']].tolist()[0]
            target_df = minidata[1:target]
            intervening_high = target_df['High'].max()
            df.at[i, 'intervening_extreme_price'] = intervening_high
    return df

In [42]:
df = add_intervening_extreme_prices(df,5)

In [43]:
df.to_csv('df.csv')